## GHG Emissions
The basis for these emission factors is the Handbook of Emission
Factors for Road Transport (HBEFA Version 3.3). (Notter et al.,2019).(3)
$$ E = A * f * l * e_f $$
Where:

E is the total CO2 emissions (gram CO2 per day)

A is the average number of vehicles of type A per day

F is the specific fuel consumption per vehicle type A (litres/km)

l is the length of the road (km)

$e_f$ is the emission factor (grams CO2/litre)

## Imports

In [92]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError


%matplotlib inline

## Global Variables

In [93]:
AADT_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/aadt.csv"
IMAGE_METADATA_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/image_metadata.csv" # Useful data: vehicle_count, vehicle_bb, road_segment_length, speed_estimate
EMISSIONS_FACTORS_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/uk_emissions_factors.csv" # emission factor (grams CO2/litre)
SPECIFIC_FUEL_CONSUMPTION_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/ground_truth_data/fuel_consumption_per_vehicle.csv" # specific fuel consumption per vehicle type A (litres/km)
ROAD_CHARACTERISTICS_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/road_characteristics.csv"

## Generic Functions

## Loading and Preprocessing Data
Multiply AADT for vehicle types by length of road segment

### AADT

In [94]:
df_aadt = pd.read_csv(AADT_PATH)
aadt = df_aadt['aadt']
df_aadt.head()

,aadt
0,18000


### Image Metadata

In [95]:
# Each row should represent a detected vehicle
df_image_metadata = pd.read_csv(IMAGE_METADATA_PATH)
df_image_metadata.head()

,x_min,y_min,x_max,y_max
0,20,50,25,55


### Road Characteristics

In [96]:
df_road_characteristics = pd.read_csv(ROAD_CHARACTERISTICS_PATH)
df_road_characteristics.head()

,road_length_km,road_width_km
0,0.235919,0.104146


In [97]:
road_segment_length = df_road_characteristics.iloc[0]['road_length_km']
road_segment_length

0.2359189453125

### Emissions Factors

In [98]:
df_emissions_factors = pd.read_csv(EMISSIONS_FACTORS_PATH)
df_emissions_factors['total'] = df_emissions_factors.sum(axis=1)
df_emissions_factors.head() # g/km

/tmp/ipykernel_519/3575893931.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_emissions_factors['total'] = df_emissions_factors.sum(axis=1)


,Vehicle Type,NOx,PM10,PM2.5,CO,VOC,NH3,SO2,Benzene,N2O,total
0,Petrol cars,0.065,0.001,0.001,0.583,0.090,0.013,0.000,0.002,0.001,0.756
1,Diesel cars,0.517,0.008,0.008,0.047,0.004,0.004,0.001,0.000,0.006,0.595
2,Petrol LGVs,0.094,0.001,0.001,1.234,0.085,0.017,0.001,0.002,0.002,1.437
3,Diesel LGVs,0.808,0.007,0.007,0.058,0.008,0.005,0.001,0.000,0.006,0.900
4,Rigid HGVs,1.428,0.020,0.020,0.438,0.039,0.009,0.002,0.000,0.031,1.987


In [99]:
# Average petrol and diesel vehicles
cars_emissions_avg = (df_emissions_factors.iloc[0]['total'] + df_emissions_factors.iloc[1]['total']) / 2
lgv_emissions_avg = (df_emissions_factors.iloc[2]['total'] + df_emissions_factors.iloc[3]['total']) / 2
hgv_bus_emissions_avg = (df_emissions_factors.iloc[4]['total'] + df_emissions_factors.iloc[5]['total'] + df_emissions_factors.iloc[6]['total']) / 3

vehicle_emissions_avg = [cars_emissions_avg, lgv_emissions_avg, hgv_bus_emissions_avg]
vehicle_emissions_avg

[0.6755, 1.1684999999999999, 2.1726666666666663]

### Specific Fuel Consumption

In [100]:
#df_specifc_fuel_consumption = pd.read_csv(SPECIFIC_FUEL_CONSUMPTION_PATH)
#df_specifc_fuel_consumption.head()

## Method

### TODO: Convert vehicle BB co-ordinates to BB areas

In [107]:
df_image_metadata['vehicle_bb_size'] = abs(df_image_metadata['x_max'] - df_image_metadata['x_min']) * abs((df_image_metadata['y_max'] - df_image_metadata['y_min']))

# convert to cm (assume GSD 0.3) => 1 pixel represents 30cm
df_image_metadata['vehicle_bb_size'] = df_image_metadata['vehicle_bb_size'].multiply(30)

df_image_metadata

,x_min,y_min,x_max,y_max,vehicle_bb_size,"vehicle_type_(0, 660)","vehicle_type_(661, 1160)","vehicle_type_(1161, 2000)",vehicle_type_emissions_avg,aadt_vehicles
0,20,50,25,55,750,0,1,0,1.1685,18000.0


### TODO: Convert vehicle BB area to vehicle type

In [102]:
# highway england dataAPI vehicle sizes in cm
vehicle_sizes = [(0, 660), (661, 1160),	(1161, 2000)]

for j in vehicle_sizes:
        df_image_metadata['vehicle_type_{}'.format(j)] = df_image_metadata['vehicle_bb_size'].between(j[0], j[1]).astype(int)

df_image_metadata

,x_min,y_min,x_max,y_max,vehicle_bb_size,"vehicle_type_(0, 660)","vehicle_type_(661, 1160)","vehicle_type_(1161, 2000)"
0,20,50,25,55,750,0,1,0


### TODO: Set emissions factors from vehicle type

In [103]:
# df_image_metadata['vehicle_type_emissions_factor']  = df_emissions_vehicle_type

df_image_metadata['vehicle_type_emissions_avg'] = (df_image_metadata['vehicle_type_(0, 660)'] * vehicle_emissions_avg[0]) + (df_image_metadata['vehicle_type_(661, 1160)'] * vehicle_emissions_avg[1]) + (df_image_metadata['vehicle_type_(1161, 2000)'] * vehicle_emissions_avg[2])
df_image_metadata

,x_min,y_min,x_max,y_max,vehicle_bb_size,"vehicle_type_(0, 660)","vehicle_type_(661, 1160)","vehicle_type_(1161, 2000)",vehicle_type_emissions_avg
0,20,50,25,55,750,0,1,0,1.1685


### TODO: Apply portions of aadt estimate to each vehicle

In [106]:
reciprocal_no_of_vehicles = 1 / len(df_image_metadata)
df_image_metadata['aadt_vehicles'] = aadt * reciprocal_no_of_vehicles
df_image_metadata

,x_min,y_min,x_max,y_max,vehicle_bb_size,"vehicle_type_(0, 660)","vehicle_type_(661, 1160)","vehicle_type_(1161, 2000)",vehicle_type_emissions_avg,aadt_vehicles
0,20,50,25,55,750,0,1,0,1.1685,18000.0


### TODO: GHG Emissions calculation

In [105]:
aadt_emissions = 0

for i in range(len(df_image_metadata)):
    aadt_emissions += road_segment_length * df_image_metadata.iloc[i]['aadt_vehicles'] * df_image_metadata.iloc[i]['vehicle_type_emissions_avg']

aadt_emissions

4962.083176757812